Rolling/Time Series Forecasting:
- features extracted with tsfresh can be useful for many different tasks, such as time series classification, compression or forecasting. This section explains how we can use the features for time series forecasting. 

- Price of a certain stock - e.g. apple for 100 time steps. now - need to build a feature based model to forecast future prices of the apple stock. 

- The rolling mechanism:

In [1]:
import pandas as pd

In [2]:
df = pd.DataFrame({
   "id": [1, 1, 1, 1, 2, 2],
   "time": [1, 2, 3, 4, 8, 9],
   "x": [1, 2, 3, 4, 10, 11],
   "y": [5, 6, 7, 8, 12, 13],
})

In [3]:
df

,id,time,x,y
0,1,1,1,5
1,1,2,2,6
2,1,3,3,7
3,1,4,4,8
4,2,8,10,12
5,2,9,11,13


Now, we can use tsfresh.utilities.dataframe_functions.roll_time_series() to get consecutive sub-time series. You could think of having a window sliding over your time series data and extracting out every data uou can see through this window. There are parameters to tune for the window.
- max_timeshift defines how large the window is at maximum. the extracted time series will have at maximum length of max_timeshift+1. (they can also be smaller, as time stamps in the beginning have less past values)
- min_timeshift defines the minimal size of each window. shorter time series (usually at the beginning will be omitted). 
- advanced: rolling direction: if you want to slide in positive (increasing sort) or negative (decreasing sort) dorection. barely need negative direction - so it might not be worth it changing the default. 

In [4]:
from tsfresh.utilities.dataframe_functions import roll_time_series

In [6]:
roll_time_series?

Signature:
roll_time_series(
    df_or_dict,
    column_id,
    column_sort=None,
    column_kind=None,
    rolling_direction=1,
    max_timeshift=None,
    min_timeshift=0,
    chunksize=None,
    n_jobs=8,
    show_warnings=False,
    disable_progressbar=False,
    distributor=None,
)
Docstring:
This method creates sub windows of the time series. It rolls the (sorted) data frames for each kind and each id
separately in the "time" domain (which is represented by the sort order of the sort column given by `column_sort`).

For each rolling step, a new id is created by the scheme ({id}, {shift}), here id is the former id of
the column and shift is the amount of "time" shifts.
You can think of it as having a window of fixed length (the max_timeshift) moving one step at a time over
your time series.
Each cut-out seen by the window is a new time series with a new identifier.

A few remarks:

 * This method will create new IDs!
 * The sign of rolling defines the direction of time rolling, a 

In [5]:
df_rolled = roll_time_series(df, column_id = "id", column_sort = "time")

Rolling: 100%|██████████| 4/4 [00:02<00:00,  1.85it/s]


the new data set consists only of values from the old dataset, but with new ids. Also the sort column values (in this case 'time') is copied. If you group by id 

In [7]:
df_rolled

,id,time,x,y
0,"(1, 1)",1,1,5
2,"(1, 2)",1,1,5
3,"(1, 2)",2,2,6
6,"(1, 3)",1,1,5
7,"(1, 3)",2,2,6
8,"(1, 3)",3,3,7
9,"(1, 4)",1,1,5
10,"(1, 4)",2,2,6
11,"(1, 4)",3,3,7
12,"(1, 4)",4,4,8


In [8]:
df

,id,time,x,y
0,1,1,1,5
1,1,2,2,6
2,1,3,3,7
3,1,4,4,8
4,2,8,10,12
5,2,9,11,13


Now - you can run the usual feature extraction procedure on the rolled data:

In [9]:
from tsfresh import extract_features

In [10]:
df_features = extract_features(df_rolled, column_id = "id", column_sort = "time")

Feature Extraction: 100%|██████████| 12/12 [00:02<00:00,  5.13it/s]


end up with features generated for each one of the parts above, which you can then use for training the forecasting model:

In [11]:
df_features

x__variance_larger_than_standard_deviation  x__has_duplicate_max  \
1 1                                         0.0                   0.0   
  2                                         0.0                   0.0   
  3                                         0.0                   0.0   
  4                                         1.0                   0.0   
2 8                                         0.0                   0.0   
  9                                         0.0                   0.0   

     x__has_duplicate_min  x__has_duplicate  x__sum_values  x__abs_energy  \
1 1                   0.0               0.0            1.0            1.0   
  2                   0.0               0.0            3.0            5.0   
  3                   0.0               0.0            6.0           14.0   
  4                   0.0               0.0           10.0           30.0   
2 8                   0.0               0.0           10.0          100.0   
  9                   0.0               0.0           21.0          221.0   

     x__mean_abs_change  x__mean_change  x__mean_second_derivative_central  \
1 1                 NaN             NaN                                NaN   
  2                 1.0             1.0                                NaN   
  3                 1.0             1.0                                0.0   
  4                 1.0             1.0                                0.0   
2 8                 NaN             NaN                                NaN   
  9                 1.0             1.0                                NaN   

     x__median  ...  y__fourier_entropy__bins_5  y__fourier_entropy__bins_10  \
1 1        1.0  ...                         NaN                          NaN   
  2        1.5  ...                   -0.000000                    -0.000000   
  3        2.0  ...                    0.693147                     0.693147   
  4        2.5  ...                    1.098612                     1.098612   
2 8       10.0  ...                         NaN                          NaN   
  9       10.5  ...                   -0.000000                    -0.000000   

     y__fourier_entropy__bins_100  y__permutation_entropy__dimension_3__tau_1  \
1 1                           NaN                                         NaN   
  2                     -0.000000                                         NaN   
  3                      0.693147                                        -0.0   
  4                      1.098612                                        -0.0   
2 8                           NaN                                         NaN   
  9                     -0.000000                                         NaN   

     y__permutation_entropy__dimension_4__tau_1  \
1 1                                         NaN   
  2                                         NaN   
  3                                         NaN   
  4                                        -0.0   
2 8                                         NaN   
  9                                         NaN   

     y__permutation_entropy__dimension_5__tau_1  \
1 1                                         NaN   
  2                                         NaN   
  3                                         NaN   
  4                                         NaN   
2 8                                         NaN   
  9                                         NaN   

     y__permutation_entropy__dimension_6__tau_1  \
1 1                                         NaN   
  2                                         NaN   
  3                                         NaN   
  4                                         NaN   
2 8                                         NaN   
  9                                         NaN   

     y__permutation_entropy__dimension_7__tau_1  \
1 1                                         NaN   
  2                                         NaN   
  3                                         NaN   